In [1]:
import sys, os
import openai
from openai import OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

from dotenv import load_dotenv
import argparse
import json
from typing import Dict, List
import logging
import GraphDB.utils as utils
from GraphDB.LegalGraphDB import LegalGraphDB
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv(verbose=True)

logging.basicConfig(level=logging.INFO)
path = os.getcwd()
root_path = os.path.dirname(path)
config_path = os.path.join(root_path, 'codes', 'configs', 'config.json')
with open(config_path, 'r') as f:
    config = json.load(f)
print(config)

dbms = LegalGraphDB(auradb=False, config=config, json_path="../graph/clause/title_embedding/01/01_law_main.json")

{'model': 'gpt-4o', 'embedding_model': 'text-embedding-3-large', 'chunk_size': 1024, 'chunk_overlap': 50, 'top_k': 5, 'prev_turns': 5, 'hop': 3, 'rag': 'Graph', 'database': 'legalgraphrag', 'query_classification_prompt_path': '../codes/GraphDB/prompt/get_query_classification.txt', 'keyword_prompt_path': '../codes/GraphDB/prompt/get_keyword_triplet.txt', 'query_keyword_prompt_path': '../codes/GraphDB/prompt/get_query_keyword.txt', 'query_answer_prompt_path': '../codes/GraphDB/prompt/get_query_answer.txt', 'generation_prompt_path': '../codes/GraphDB/prompt/generation.txt', 'threshold': 0.4, 'max_tokens': 1024}
## Settings ##
## clause type :  Clause_01_law_main
## title_prefix :  Title_01_law_main_
## 서버와의 연결이 성공적으로 확인되었습니다.


# 4. Retrieval (쿼리, 시작점, 끝점 가정) 
    
    → 시작점 - 끝점 사이의 node들을 반환 (Closed Loop 찾기) 
    
    (n이 클수록 풍부성 증가) 
    
    - 시작 - (노드) - 끝 : 1 path
    - 시작 - (노드) - (노드)- 끝 : 2 path
    - 시작 - (노드) - … (노드) -끝 : n path

In [2]:
query = "증권신고서를 부실기재했을 때 법인의 대표자에게 부과되는 과징금은 얼마인가?"
query_emb = utils.text_embed(query)

## client loaded ##
## config file loaded ##
## Settings ##
## clause type :  Clause_01_law_main
## title_prefix :  Title_01_law_main_
## 서버와의 연결이 성공적으로 확인되었습니다.
## config file loaded ##


In [3]:
query_keywords= utils.extract_keyword(query)


# keywords_graph = ["공개매수신고서", "증권신고서", "일괄신고서"]
# for i in range(len(keywords_graph)):
#     query_keywords[0].append(keywords_graph[i])
print(query_keywords)


(['증권신고서', '부실기재', '법인 대표자', '과징금'], ['증권신고서', '부실기재', '법인', '대표자', '과징금'])


In [4]:
from itertools import combinations

keyword_comb = list(combinations(query_keywords[0], 2))
keyword_comb_text = [' '.join(pair) for pair in keyword_comb]

# keyword_comb_text = query_keywords[0] + keyword_comb_text
print(len(keyword_comb_text), keyword_comb_text)



6 ['증권신고서 부실기재', '증권신고서 법인 대표자', '증권신고서 과징금', '부실기재 법인 대표자', '부실기재 과징금', '법인 대표자 과징금']


# 단순 Top k (VectorRAG)

In [5]:
results= dbms.get_top_k_clause_nodes(query_emb, k = 15)
for result in results:
    print(result)

##<제429조제4항>, 제8편 감독 및 처분 제4장 과징금 제2절 예탁관련제도 제2관 투자익명조합, 공시위반에 대한 과징금
금융위원회는 제147조제1항에 따라 보고를 하여야 할 자가 다음 각 호의 어느 하나에 해당하는 경우에는 같은 항에 따른 주권상장법인이 발행한 주식의 시가총액(대통령령으로 정하는 방법에 따라 산정된 금액으로 한다)의 10만분의 1(5억원을 초과하는 경우에는 5억원)을 초과하지 아니하는 범위에서 과징금을 부과할 수 있다.<개정 2013. 5. 28.>
1. 제147조제1항ㆍ제3항 또는 제4항을 위반하여 보고를 하지 아니한 경우
2. 제147조에 따른 보고서류 또는 제151조제2항에 따른 정정보고서 중 대통령령으로 정하는 중요한 사항에 관하여 거짓의 기재 또는 표시를 하거나 중요한 사항을 기재 또는 표시하지 아니한 경우
##<제349조제2항>, 제6편 금융투자업관계기관 제4장 종합금융회사 제2절 예탁관련제도 제2관 투자익명조합, 과징금
금융위원회는 종합금융회사가 제347조제1항 또는 제2항을 위반한 경우에는 그 종합금융회사에 대하여 다음 각 호의 구분에 따른 범위에서 과징금을 부과할 수 있다.<신설 2017. 4. 18.>
1. 제347조제1항을 위반한 경우: 부동산 취득금액의 100분의 30
2. 제347조제2항을 위반한 경우: 한도를 초과한 업무용 부동산 취득금액의 100분의 30
##<제162조제1항>, 제3편 증권의 발행 및 유통 제3장 상장법인의 사업보고서 등 제3절 의결권 대리행사의 권유 제한 제4관 신탁업자의 영업행위 규칙, 거짓의 기재 등에 의한 배상책임
제159조제1항의 사업보고서ㆍ반기보고서ㆍ분기보고서ㆍ주요사항보고서(이하 "사업보고서등"이라 한다) 및 그 첨부서류(회계감사인의 감사보고서는 제외한다) 중 중요사항에 관하여 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 사업보고서 제출대상법인이 발행한 증권(그 증권과 관련된 증권예탁증권, 그 밖에 대통령령으로 정하는 증권을 포함한다. 이하 이 조에서 같

In [6]:

final_start_nodes = [] 
for i in range(len(keyword_comb_text)):
    print("========================="*2)
    node_id_list = [] 
    print(keyword_comb_text[i])
    keyword_emb = utils.text_embed(keyword_comb_text[i])
    results= dbms.get_top_k_clause_nodes(keyword_emb, k = 1)
    for result in results:
        node_id = int(result['node_id'].split(':')[-1])
        node_id_list.append(dbms.get_top_k_in_same_index(query_emb, node_id, k=1))
    sorted_data = sorted(node_id_list, key=lambda x: x[0]['similarity'], reverse=True)
    print(sorted_data[0])
    final_start_nodes.append(sorted_data[0][0]['node_id'])
#distinct 구분 
final_start_nodes = list(set(final_start_nodes))
print(final_start_nodes)

증권신고서 부실기재
##<제125조제1항>, 제3편 증권의 발행 및 유통 제1장 증권신고서 제2절 금융투자업자별 영업행위 규칙 제4관 신탁업자의 영업행위 규칙, 거짓의 기재 등으로 인한 배상책임
증권신고서(정정신고서 및 첨부서류를 포함한다. 이하 이 조에서 같다)와 투자설명서(예비투자설명서 및 간이투자설명서를 포함한다. 이하 이 조에서 같다) 중 중요사항에 관하여 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 증권의 취득자가 손해를 입은 경우에는 다음 각 호의 자는 그 손해에 관하여 배상의 책임을 진다. 다만, 배상의 책임을 질 자가 상당한 주의를 하였음에도 불구하고 이를 알 수 없었음을 증명하거나 그 증권의 취득자가 취득의 청약을 할 때에 그 사실을 안 경우에는 배상의 책임을 지지 아니한다. <개정 2009. 2. 3., 2013. 5. 28.>
1. 그 증권신고서의 신고인과 신고 당시의 발행인의 이사(이사가 없는 경우 이에 준하는 자를 말하며, 법인의 설립 전에 신고된 경우에는 그 발기인을 말한다)
2. 「상법」 제401조의2제1항 각 호의 어느 하나에 해당하는 자로서 그 증권신고서의 작성을 지시하거나 집행한 자
3. 그 증권신고서의 기재사항 또는 그 첨부서류가 진실 또는 정확하다고 증명하여 서명한 공인회계사ㆍ감정인 또는 신용평가를 전문으로 하는 자 등(그 소속단체를 포함한다) 대통령령으로 정하는 자
4. 그 증권신고서의 기재사항 또는 그 첨부서류에 자기의 평가ㆍ분석ㆍ확인 의견이 기재되는 것에 대하여 동의하고 그 기재내용을 확인한 자
5. 그 증권의 인수인 또는 주선인(인수인 또는 주선인이 2인 이상인 경우에는 대통령령으로 정하는 자를 말한다)
6. 그 투자설명서를 작성하거나 교부한 자
7. 매출의 방법에 의한 경우 매출신고 당시의 매출인
[{'node_id': 2185, 'index': '제125조제1항', 'name': '거짓의 기재 등으로 인한 배상책임', 'similarity': 0.56, 'text': '증권신고서(정정

###  2개 일 때 (시작)-(끝)

In [7]:
from neo4j import GraphDatabase
from collections import Counter

dbms.find_path_node(final_start_nodes) 

##<제429조제4항>, 제8편 감독 및 처분 제4장 과징금 제2절 예탁관련제도 제2관 투자익명조합, 공시위반에 대한 과징금
금융위원회는 제147조제1항에 따라 보고를 하여야 할 자가 다음 각 호의 어느 하나에 해당하는 경우에는 같은 항에 따른 주권상장법인이 발행한 주식의 시가총액(대통령령으로 정하는 방법에 따라 산정된 금액으로 한다)의 10만분의 1(5억원을 초과하는 경우에는 5억원)을 초과하지 아니하는 범위에서 과징금을 부과할 수 있다.<개정 2013. 5. 28.>
1. 제147조제1항ㆍ제3항 또는 제4항을 위반하여 보고를 하지 아니한 경우
2. 제147조에 따른 보고서류 또는 제151조제2항에 따른 정정보고서 중 대통령령으로 정하는 중요한 사항에 관하여 거짓의 기재 또는 표시를 하거나 중요한 사항을 기재 또는 표시하지 아니한 경우
##<제429조제1항>, 제8편 감독 및 처분 제4장 과징금 제2절 예탁관련제도 제2관 투자익명조합, 공시위반에 대한 과징금
금융위원회는 제125조제1항 각 호의 어느 하나에 해당하는 자가 다음 각 호의 어느 하나에 해당하는 경우에는 증권신고서상의 모집가액 또는 매출가액의 100분의 3(20억원을 초과하는 경우에는 20억원)을 초과하지 아니하는 범위에서 과징금을 부과할 수 있다. <개정 2008. 2. 29.>
1. 제119조, 제122조 또는 제123조에 따른 신고서ㆍ설명서, 그 밖의 제출서류 중 중요사항에 관하여 거짓의 기재 또는 표시를 하거나 중요사항을 기재 또는 표시하지 아니한 때
2. 제119조, 제122조 또는 제123조에 따른 신고서ㆍ설명서, 그 밖의 제출서류를 제출하지 아니한 때
##<제125조제1항>, 제3편 증권의 발행 및 유통 제1장 증권신고서 제2절 금융투자업자별 영업행위 규칙 제4관 신탁업자의 영업행위 규칙, 거짓의 기재 등으로 인한 배상책임
증권신고서(정정신고서 및 첨부서류를 포함한다. 이하 이 조에서 같다)와 투자설명서(예비투자설명서 및 간이투자설명서를 포함한다. 이하 이 조에서 같다) 중 중요사항에

[{'node_id': '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3520',
  'index': '제429조제4항',
  'labels': 'Clause_01_law_main',
  'text': '금융위원회는 제147조제1항에 따라 보고를 하여야 할 자가 다음 각 호의 어느 하나에 해당하는 경우에는 같은 항에 따른 주권상장법인이 발행한 주식의 시가총액(대통령령으로 정하는 방법에 따라 산정된 금액으로 한다)의 10만분의 1(5억원을 초과하는 경우에는 5억원)을 초과하지 아니하는 범위에서 과징금을 부과할 수 있다.<개정 2013. 5. 28.>\n1. 제147조제1항ㆍ제3항 또는 제4항을 위반하여 보고를 하지 아니한 경우\n2. 제147조에 따른 보고서류 또는 제151조제2항에 따른 정정보고서 중 대통령령으로 정하는 중요한 사항에 관하여 거짓의 기재 또는 표시를 하거나 중요한 사항을 기재 또는 표시하지 아니한 경우',
  'reference': '제8편 감독 및 처분 제4장 과징금 제2절 예탁관련제도 제2관 투자익명조합',
  'name': '공시위반에 대한 과징금'},
 {'node_id': '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3517',
  'index': '제429조제1항',
  'labels': 'Clause_01_law_main',
  'text': '금융위원회는 제125조제1항 각 호의 어느 하나에 해당하는 자가 다음 각 호의 어느 하나에 해당하는 경우에는 증권신고서상의 모집가액 또는 매출가액의 100분의 3(20억원을 초과하는 경우에는 20억원)을 초과하지 아니하는 범위에서 과징금을 부과할 수 있다. <개정 2008. 2. 29.>\n1. 제119조, 제122조 또는 제123조에 따른 신고서ㆍ설명서, 그 밖의 제출서류 중 중요사항에 관하여 거짓의 기재 또는 표시를 하거나 중요사항을 기재 또는 표시하지 아니한 때\n2. 제119조, 제122조 또는 제123조에 따른 신고서ㆍ설명서, 그 

### n개 일 때 


In [8]:
cypher = """

// 노드 리스트를 ID 배열로 정의
WITH ['4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2185', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2167', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2172', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2176', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2209', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2167', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2160', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2185', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2195', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3517', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2282', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2197', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2176', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2415', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3655', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2160', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2271', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:754', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:1593', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3585', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3520', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2160', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:2271', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:754', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3517', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:1595', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3560', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3600', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:1595', '4:cc44eefc-a31b-4cfb-9ee0-f031598dcc09:3519'] AS node_ids

// 노드 리스트에서 각 노드를 찾고, 이들 간 모든 경로를 탐색
UNWIND node_ids AS node_id
MATCH (n {element_id: node_id})

// n개의 노드 간 경로 탐색, 순환 경로 확인
WITH collect(n) AS nodes
MATCH p = (nodes[0])-[*]-(nodes[-1]) // 첫 번째와 마지막 노드를 잇는 경로를 탐색
WHERE ALL(i IN range(0, size(nodes) - 2) WHERE (nodes[i])-[*]-(nodes[i + 1])) 
AND (nodes[0])-[*]-(nodes[0]) // 순환 경로가 형성되어야 함

// 결과 반환: 경로 및 중간 노드 수
RETURN p, size(nodes(p)) - size(nodes) AS intermediate_node_count
LIMIT 1


"""